<a href="https://colab.research.google.com/github/James-R-Chapman/Kia-Invoices/blob/main/Kia_Invoices_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install camelot-py[cv]
#!apt-get install ghostscript
#!sudo apt install tesseract-ocr
!pip install openpyxl
!pip install PyPDF2
!pip install PyPDF4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.3/248.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 18.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyPDF4: filename=PyPDF4-1.27.0-py3-none-any.whl size=61246 sh

In [ ]:
import csv
import glob
import re
import camelot
import pandas as pd
from pdfminer.high_level import extract_text
from google.colab import files
import openpyxl
from google.colab import drive
import os
drive.mount('/content/drive')

# Upload the PDF file
#uploaded = files.upload()

# Get the path of the uploaded file
#pdf_path = list(uploaded.keys())[0]
#pdf_path = '/content/Batch Report (39).pdf'

pdf_folder = '/content/drive/MyDrive/Courage Kia/Kia Invoices'
csv_file_path = '/content/drive/MyDrive/Courage Kia/Invoice Rec/parts_invoices.csv'
# Check if parts_invoices.csv file already exists
if os.path.isfile(csv_file_path):
    skip = True
    print(f"{csv_file_path} already exists. Skip is set to {skip}.")
else:
    skip = False
    print(f"{csv_file_path} does not exist. Skip is set to {skip}.")

for pdf_path in glob.glob(pdf_folder + '/*.pdf'):

    if not skip:
        print(f"Reading file: {pdf_path}")
        
        # Extract the text from the PDF file using pdfminer
        text = extract_text(pdf_path)

        # Split the text into lines and remove any empty lines
        lines = [line.strip() for line in text.split('\n') if line.strip()]

        # Write the lines to the CSV file in append mode
        with open(csv_file_path, 'a', newline='') as csv_file:
            writer = csv.writer(csv_file)
            writer.writerows([line.split() for line in lines])

#Get invoice number
match_invoice = re.search(r'[C,P][A-Z]\d{8}', text)
#Get invoice date
match_date = re.search(r'\d{1,2}/\d{1,2}/\d{2}', text)
#Get invoice page number
match_page = re.search(r'PAGE:\s*1', text)
#Get order number
match_order = re.search(r'ORDER NO: (\w+) ORDER DATE:', text)
#Get line, part_no, advise_code, qty_order, qty_shipped, cost, price
match = re.search(r'^(?P<line>\d+)\s+(?P<part_no>\d+\S+)\s+(?P<advise_code>\S+)\s+(?P<qty_order>\d+)\s+(?P<qty_shipped>\d+)\s+(?P<cost>[\d.]+)\s+(?P<price>[\d.]+)\s+\S+$', text)
if match:
    line_num = match.group('line')
    part_no = match.group('part_no')
    advise_code = match.group('advise_code')
    qty_order = match.group('qty_order')
    qty_shipped = match.group('qty_shipped')
    cost = match.group('cost')
    price = match.group('price')
match = re.search(r'Core Charge\s+(\d+)\s+([\d.]+)\s+([\d.]+)', text)
if match:
    part_no = "Core Charge"
    qty_ship = match.group(1)
    cost = match.group(2)
    price = match.group(3)
match = re.search(r'\*\*\*Core Charge Amount\*\*\*\s+([\d.]+)', text)
if match:
    core_charge_amount = match.group(1)
    core_cost = core_charge_amount
#Get subtotal, freight, handling, misc, tax, invoice_total
subtotal_match = re.search(r'SUB TOTAL (\d+\.\d+)', text)
freight_match = re.search(r'FREIGHT:\s*(\d+\.\d*)', text)
handling_match = re.search(r'HANDLING:\s*(\d+\.\d*)', text)
misc_match = re.search(r'MISC:\s*(\d+\.\d*)', text)
tax_match = re.search(r'TAX:\s*(\d+\.\d*)', text)
invoice_total_match = re.search(r'INVOICE TOTAL (\d+\.\d+)', text)

subtotal = float(subtotal_match.group(1)) if subtotal_match else 0
freight = float(freight_match.group(1)) if freight_match else 0
handling = float(handling_match.group(1)) if handling_match else 0
misc = float(misc_match.group(1)) if misc_match else 0
tax = float(tax_match.group(1)) if tax_match else 0
invoice_total = float(invoice_total_match.group(1)) if invoice_total_match else 0

print(subtotal, freight, handling, misc, tax, invoice_total)

In [13]:
import re
import PyPDF2
import openpyxl
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment

pdf_path = '/content/Invoices.pdf'

# Define a function to extract invoice info from a single page
def extract_invoice_info(text):
    # Get invoice number
    match_invoice = re.search(r'[C,P][A-Z]\d{8}', text)
    invoice_num = match_invoice.group() if match_invoice else None

    # Get invoice date
    match_date = re.search(r'\d{1,2}/\d{1,2}/\d{2}', text)
    invoice_date = match_date.group() if match_date else None

    # Get invoice page number
    match_page = re.search(r'PAGE:\s*(\d+)', text)
    invoice_page = match_page.group(1) if match_page else None

    # Get order number
    match_order = re.search(r'ORDER NO: (\w+) ORDER DATE:', text)
    order_num = match_order.group(1) if match_order else None

    # Get line, part_no, advise_code, qty_order, qty_shipped, cost, price
    match = re.search(r'^(?P<line>\d+)\s+(?P<part_no>\d+\S+)\s+(?P<advise_code>\S+)\s+(?P<qty_order>\d+)\s+(?P<qty_shipped>\d+)\s+(?P<cost>[\d.]+)\s+(?P<price>[\d.]+)\s+\S+$', text)
    if match:
        line_num = match.group('line')
        part_no = match.group('part_no').replace(' ', '')
        advise_code = match.group('advise_code')
        qty_order = match.group('qty_order')
        qty_shipped = match.group('qty_shipped')
        cost = match.group('cost')
        price = match.group('price')
    else:
        line_num = None
        part_no = None
        advise_code = None
        qty_order = None
        qty_shipped = None
        cost = None
        price = None

    # Get core charge amount
    core_charge_amount = None
    core_cost = None
    core_match = re.search(r'Core Charge Amount\*\*\*\s+([\d.]+)', text)
    if core_match:
        core_charge_amount = core_match.group(1)
        core_cost = core_charge_amount

    # Get subtotal, freight, handling, misc, tax, invoice_total
    subtotal_match = re.search(r'SUB TOTAL (\d+\.\d+)', text)
    freight_match = re.search(r'FREIGHT:\s*(\d+\.\d*)', text)
    handling_match = re.search(r'HANDLING:\s*(\d+\.\d*)', text)
    misc_match = re.search(r'MISC:\s*(\d+\.\d*)', text)
    tax_match = re.search(r'TAX:\s*(\d+\.\d*)', text)
    invoice_total_match = re.search(r'INVOICE TOTAL (\d+\.\d+)', text)
    subtotal = float(subtotal_match.group(1)) if subtotal_match else 0
    freight = float(freight_match.group(1)) if freight_match else 0
    handling = float(handling_match.group(1)) if handling_match else 0
    misc = float(misc_match.group(1)) if misc_match else 0
    tax = float(tax_match.group(1)) if tax_match else 0
    invoice_total = float(invoice_total_match.group(1)) if invoice_total_match else 0

    return {
        "invoice_num": invoice_num,
        "invoice_date": invoice_date,
        "invoice_page": invoice_page,
        "order_num": order_num,
        "line_num": line_num,
        "part_no": part_no,
        "advise_code": advise_code,
        "qty_order": qty_order,
        "qty_shipped": qty_shipped,
        "cost": cost,
        "price": price,
        "core_charge_amount": core_charge_amount,
        "core_cost": core_cost,
        "subtotal": subtotal,
        "freight": freight,
        "handling": handling,
        "misc": misc,
        "tax": tax,
        "invoice_total": invoice_total
    }


# Open the PDF file and read its contents
with open(pdf_path, 'rb') as f:
    reader = PyPDF2.PdfReader(f)
    num_pages = len(reader.pages)
    pages_text = []
    for i in range(num_pages):
        page = reader.pages[i]
        text = page.extract_text()
        pages_text.append(text)

# Extract invoice information from each page
invoice_info_list = []
for text in pages_text:
    invoice_info = extract_invoice_info(text)
    invoice_info_list.append(invoice_info)

# Create a new workbook and add a worksheet
workbook = openpyxl.Workbook()
worksheet = workbook.active

headers = [
    "Invoice Number",
    "Invoice Date",
    "Invoice Page",
    "Order Number",
    "Line Number",
    "Part Number",
    "Advise Code",
    "Quantity Ordered",
    "Quantity Shipped",
    "Cost",
    "Price",
    "Core Charge Amount",
    "Subtotal",
    "Freight",
    "Handling",
    "Misc",
    "Tax",
    "Invoice Total"
]

worksheet.append(headers)
# Write the invoice data to the worksheet
for invoice_info in invoice_info_list:
    worksheet.append([
        invoice_info.get('invoice_num', ''),
        invoice_info.get('invoice_date', ''),
        invoice_info.get('page_num', ''),
        invoice_info.get('order_num', ''),
        invoice_info.get('line_num', ''),
        invoice_info.get('part_no', ''),
        invoice_info.get('advise_code', ''),
        invoice_info.get('qty_order', ''),
        invoice_info.get('qty_shipped', ''),
        invoice_info.get('cost', ''),
        invoice_info.get('price', ''),
        invoice_info.get('core_charge_amount', ''),
        invoice_info.get('subtotal', ''),
        invoice_info.get('freight', ''),
        invoice_info.get('handling', ''),
        invoice_info.get('misc', ''),
        invoice_info.get('tax', ''),
        invoice_info.get('invoice_total', ''),
    ])

# Save the workbook to a file
workbook.save('invoices.xlsx')